In [33]:
# interet de energy star score
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_boston
from sklearn.datasets import make_regression
from sklearn import set_config 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from sklearn.preprocessing import scale

import joblib
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

In [34]:
new_data2 = joblib.load('new_data2.joblib')
new_data2.head(3)

,BuildingType,PrimaryPropertyType,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,ENERGYSTARScore
0,NonResidential,Hotel,downtown,47.61220,-122.33799,1927,1.0,12,88434,7226362.5,2003882.0,3946027.0,1276453.0,249.98,60.0
1,NonResidential,Hotel,downtown,47.61317,-122.33393,1996,1.0,11,103566,8387933.0,0.0,3242851.0,5145082.0,295.86,61.0
2,NonResidential,Hotel,downtown,47.61393,-122.33810,1969,1.0,41,956110,72587024.0,21566554.0,49526664.0,1493800.0,2089.28,43.0


# one hot encoding

In [35]:
def get_ohe(df, column_name):
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(df[[column_name]])
    temp_df = pd.DataFrame(data=ohe.transform(df[[column_name]]), columns=ohe.get_feature_names_out())
    df.drop(columns=[column_name], axis=1, inplace=True)
    df = pd.concat([df.reset_index(drop=True), temp_df], axis=1)
    return df

new_data2 = get_ohe(new_data2, 'BuildingType')
new_data2 = get_ohe(new_data2, 'PrimaryPropertyType')
new_data2 = get_ohe(new_data2, 'Neighborhood')

## Total GHG Emissions

In [36]:
y = new_data2[['TotalGHGEmissions']]

#Je sélectionne toutes les colonnes sauf la dernière (qui est ma cible) pour les mettre dans la variable X
X = new_data2.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions', 'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)'], axis=1)

In [37]:
std_scale = preprocessing.StandardScaler().fit(X)
X_std = std_scale.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2)

In [38]:
#set_config(print_changed_only=False) 
 
#rfr = RandomForestRegressor(criterion='absolute_error')
#print(rfr)
# paramètre par défaut#
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='absolute_error',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False) 

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='absolute_error',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [39]:
max_features_range  = ['sqrt', 'log2']
max_depth_range = [25, 50, 75]
min_samples_split_range = [2, 5, 10]
bootstrap_range = [True, False]
n_estimators_range = [100, 150, 200]
min_samples_leaf_range = [1,2,5,10]

param_rfr = {'max_features' : max_features_range,
             'max_depth': max_depth_range,
             'min_samples_split': min_samples_split_range,
             'bootstrap' : bootstrap_range,
             'min_samples_leaf': min_samples_leaf_range,
             'n_estimators' : n_estimators_range
             }

In [40]:
score = 'neg_mean_absolute_error'

grid_pred = GridSearchCV(RandomForestRegressor(criterion='absolute_error'),
        param_grid=param_rfr,
        cv=5,
        scoring=score)
                                    
grid_pred.fit(X_train, y_train)

/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/Users/yixuanzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y t

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='absolute_error',
                                             max_depth=None, max_features=1.0,
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             n_jobs=None,
             param_grid={'bootstrap': [True, False], 'max_depth': [25, 50, 75],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 5, 10],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 150, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=0)

In [41]:
grid_pred.best_params_

{'bootstrap': False,
 'max_depth': 25,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 100}

In [42]:
best_grid = grid_pred.best_estimator_
y_test_pred_cv = best_grid.predict(X_test)

In [43]:
best_grid

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0,
                      criterion='absolute_error', max_depth=25,
                      max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_samples_leaf=2, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [46]:
mae = mean_absolute_error(y_test, y_test_pred_cv)
mse = mean_squared_error(y_test, y_test_pred_cv)
mape = mean_absolute_percentage_error(y_test, y_test_pred_cv)
r2 = r2_score(y_test,y_test_pred_cv)

In [49]:
print("MAE : " , mae)
print("R2 : ", r2)
print("MSE : ", mse)
print("MAPE : ", mape)


MAE :  102.79750388127856
R2 :  0.5662559049434908
MSE :  165610.6837082478
MAPE :  1.352116293845732
